In [71]:
import os
import json
import yaml
import openai
import prompts
import workout
from rich import print
from langchain.output_parsers import PydanticOutputParser
from dotenv import load_dotenv, find_dotenv
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import pandas as pd
from langchain.schema import AIMessage, HumanMessage, SystemMessage

load_dotenv(find_dotenv())

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [117]:
chat = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.1,
    openai_api_key=openai_api_key,
    verbose=True,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [107]:
with open('response.txt', 'r') as f:
    steps = f.read()

In [108]:
steps_yaml = yaml.safe_load(steps)["Steps"]

steps_yaml

{'summary': [{'Age': '18-24'},
  {'Gender': 'Male'},
  {'Height': '160.0 cms'},
  {'Weight': '60.0 kgs'},
  {'Fitness level': 'Intermediate'},
  {'Structured strength training program': True},
  {'Days per week for strength training': '4 days'},
  {'Primary goal': 'Improve strength'},
  {'Gym access': True},
  {'Weakest muscle group': 'Arms'},
  {'Strength training program preference': 'Body part splits'},
  {'Bodyweight exercises or free weights': 'Free weights'},
  {'Previous injuries or medical conditions': 'None'},
  {'Preferred duration for each workout session': '60-90 minutes'},
  {'Equipment access': 'Kettlebells, Resistance bands'},
  {'Workout program duration': '3 months'}],
 'steps': [{'step': "Calculate the client's Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure (TDEE) to determine the daily caloric intake."},
  {'step': 'Design a 4-day workout program with a focus on the arms, incorporating free weight exercises such as bicep curls, tricep extensions, and h

In [109]:
summary_gpt_response = yaml.safe_dump(steps_yaml['summary'])
steps_gpt_response = yaml.safe_dump(steps_yaml['steps'])
print(summary_gpt_response)
print(steps_gpt_response)

- Age: 18-24
- Gender: Male
- Height: 160.0 cms
- Weight: 60.0 kgs
- Fitness level: Intermediate
- Structured strength training program: true
- Days per week for strength training: 4 days
- Primary goal: Improve strength
- Gym access: true
- Weakest muscle group: Arms
- Strength training program preference: Body part splits
- Bodyweight exercises or free weights: Free weights
- Previous injuries or medical conditions: None
- Preferred duration for each workout session: 60-90 minutes
- Equipment access: Kettlebells, Resistance bands
- Workout program duration: 3 months

- step: Calculate the client's Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure
    (TDEE) to determine the daily caloric intake.
- step: Design a 4-day workout program with a focus on the arms, incorporating free
    weight exercises such as bicep curls, tricep extensions, and hammer curls.
- step: Incorporate compound exercises such as bench press, deadlifts, and squats
    to improve overall strength.
- step: Include kettlebell and resistance band exercises to add variety to the workout
    program.
- step: Ensure that the workout program is progressive, increasing the weight and
    intensity gradually over the 3-month duration.
- step: Provide the client with a detailed workout plan, including the number of sets,
    reps, and rest periods for each exercise.
- step: Monitor the client's progress regularly and make adjustments to the workout
    program as necessary.

In [110]:
plan_generator_message = HumanMessagePromptTemplate.from_template(
    template="""You will now be provided with a summary of the CLIENT's information and the steps to be taken to create a highly personalized workout plan.
    
    ```
    SUMMARY:
    {summary_gpt_response}
    ```

    ```
    STEPS: 
    {steps_gpt_response}
    ```

    Generate a highly personalized workout plan for the CLIENT with ONLY the provided information."""
)

In [111]:
workout_prompt = ChatPromptTemplate.from_messages(
    [prompts.system_message, plan_generator_message, prompts.schema_message]
)

messages = workout_prompt.format_prompt(
    output_schema=prompts.WORKOUT_SCHEMA,
    example_response=prompts.EXAMPLE_RESPONSE,
    summary_gpt_response=summary_gpt_response,
    steps_gpt_response=steps_gpt_response,
)

print(chat.get_num_tokens_from_messages(messages.to_messages()))
print(messages.to_string())

917

System: Your task is to act as a personal fitness trainer for a CLIENT. 
    You will be provided with information about your CLIENT's personal information, their fitness history, their 
goals, and any physical constraints they may have. 
    You have to prepare a detailed workout plan for the CLIENT, including everything essential for physical 
fitness.
    You will interact with the CLIENT only once currently, so do not think of steps that will be taken in the 
future.
    Ensure that you STRICTLY adhere to the number of days and weeks the CLIENT is willing to do their workout.
Human: You will now be provided with a summary of the CLIENT's information and the steps to be taken to create a 
highly personalized workout plan.
    
    ```
    SUMMARY:
    - Age: 18-24
- Gender: Male
- Height: 160.0 cms
- Weight: 60.0 kgs
- Fitness level: Intermediate
- Structured strength training program: true
- Days per week for strength training: 4 days
- Primary goal: Improve strength
- Gym access: true
- Weakest muscle group: Arms
- Strength training program preference: Body part splits
- Bodyweight exercises or free weights: Free weights
- Previous injuries or medical conditions: None
- Preferred duration for each workout session: 60-90 minutes
- Equipment access: Kettlebells, Resistance bands
- Workout program duration: 3 months

    ```

    ```
    STEPS: 
    - step: Calculate the client's Basal Metabolic Rate (BMR) and Total Daily Energy Expenditure
    (TDEE) to determine the daily caloric intake.
- step: Design a 4-day workout program with a focus on the arms, incorporating free
    weight exercises such as bicep curls, tricep extensions, and hammer curls.
- step: Incorporate compound exercises such as bench press, deadlifts, and squats
    to improve overall strength.
- step: Include kettlebell and resistance band exercises to add variety to the workout
    program.
- step: Ensure that the workout program is progressive, increasing the weight and
    intensity gradually over the 3-month duration.
- step: Provide the client with a detailed workout plan, including the number of sets,
    reps, and rest periods for each exercise.
- step: Monitor the client's progress regularly and make adjustments to the workout
    program as necessary.

    ```

    Generate a highly personalized workout plan for the CLIENT with ONLY the provided information.
Human: 
The output should be formatted in a way that conforms to the given schema below. As an example, a valid response 
is: 

WorkoutPlan:
  wks:
    - wk_range: Week 1-2
      days:
        - num: 1
          focus: Body Part A
          exs:
            - name: Exercise A1
              type: Type A
              sets: 3
              reps: 10
            - name: Exercise A3
              type: Type B
              dur:
                val: 30
                unit: seconds
        - num: 2
          focus: Body Part B
          exs:
            - name: Exercise B1
              type: Type A
              sets: 4
              reps: 8
            - name: Exercise B2
              type: Type A
              sets: 3
              reps: 10
  notes:
    - content: Note 1
    - content: Note 2


Here is the output schema:
```

WorkoutPlan:
  wks:
    - wk_range: Week or range of weeks in the plan (e.g., Week 1-2).
    days:
      - num: Day number within the workout plan.
      - focus: Primary muscle group or activity targeted on this day.
      exs:
        - name: Name of the exercise.
        - type: Category of the exercise (warmup, cardio, stretching, or strength).
        - sets: Number of sets to perform.
        - reps: Number of repetitions per set.
        dur:
          val: Duration value of the exercise.
          unit: Time unit for the duration (e.g., seconds, minutes).
  notes:
    - content: Additional information, tips, or instructions related to the workout plan as individual items.

```

In [118]:
response = chat(messages.to_messages())

WorkoutPlan:
  wks:
    - wk_range: Week 1-4
      days:
        - num: 1
          focus: Arms
          exs:
            - name: Bicep Curls
              type: Strength
              sets: 3
              reps: 10
            - name: Tricep Extensions
              type: Strength
              sets: 3
              reps: 10
            - name: Hammer Curls
              type: Strength
              sets: 3
              reps: 10
        - num: 2
          focus: Chest and Back
          exs:
            - name: Bench Press
              type: Strength
              sets: 4
              reps: 8
            - name: Deadlifts
              type: Strength
              sets: 4
              reps: 8
        - num: 3
          focus: Legs
          exs:
            - name: Squats
              type: Strength
              sets: 4
              reps: 8
            - name: Leg Press
              type: Strength
              sets: 4
              reps: 8
        - num: 4
          focus: S

In [119]:
response_json = yaml.safe_load(response.content)

print(response_json)

{
    'WorkoutPlan': {
        'wks': [
            {
                'wk_range': 'Week 1-4',
                'days': [
                    {
                        'num': 1,
                        'focus': 'Arms',
                        'exs': [
                            {'name': 'Bicep Curls', 'type': 'Strength', 'sets': 3, 'reps': 10},
                            {'name': 'Tricep Extensions', 'type': 'Strength', 'sets': 3, 'reps': 10},
                            {'name': 'Hammer Curls', 'type': 'Strength', 'sets': 3, 'reps': 10}
                        ]
                    },
                    {
                        'num': 2,
                        'focus': 'Chest and Back',
                        'exs': [
                            {'name': 'Bench Press', 'type': 'Strength', 'sets': 4, 'reps': 8},
                            {'name': 'Deadlifts', 'type': 'Strength', 'sets': 4, 'reps': 8}
                        ]
                    },
                    {
                        'num': 3,
                        'focus': 'Legs',
                        'exs': [
                            {'name': 'Squats', 'type': 'Strength', 'sets': 4, 'reps': 8},
                            {'name': 'Leg Press', 'type': 'Strength', 'sets': 4, 'reps': 8}
                        ]
                    },
                    {
                        'num': 4,
                        'focus': 'Shoulders and Core',
                        'exs': [
                            {'name': 'Kettlebell Swings', 'type': 'Strength', 'sets': 3, 'reps': 10},
                            {'name': 'Resistance Band Rows', 'type': 'Strength', 'sets': 3, 'reps': 10}
                        ]
                    }
                ]
            },
            {
                'wk_range': 'Week 5-8',
                'days': [
                    {
                        'num': 1,
                        'focus': 'Arms',
                        'exs': [
                            {'name': 'Bicep Curls', 'type': 'Strength', 'sets': 4, 'reps': 10},
                            {'name': 'Tricep Extensions', 'type': 'Strength', 'sets': 4, 'reps': 10},
                            {'name': 'Hammer Curls', 'type': 'Strength', 'sets': 4, 'reps': 10}
                        ]
                    },
                    {
                        'num': 2,
                        'focus': 'Chest and Back',
                        'exs': [
                            {'name': 'Bench Press', 'type': 'Strength', 'sets': 5, 'reps': 8},
                            {'name': 'Deadlifts', 'type': 'Strength', 'sets': 5, 'reps': 8}
                        ]
                    },
                    {
                        'num': 3,
                        'focus': 'Legs',
                        'exs': [
                            {'name': 'Squats', 'type': 'Strength', 'sets': 5, 'reps': 8},
                            {'name': 'Leg Press', 'type': 'Strength', 'sets': 5, 'reps': 8}
                        ]
                    },
                    {
                        'num': 4,
                        'focus': 'Shoulders and Core',
                        'exs': [
                            {'name': 'Kettlebell Swings', 'type': 'Strength', 'sets': 4, 'reps': 10},
                            {'name': 'Resistance Band Rows', 'type': 'Strength', 'sets': 4, 'reps': 10}
                        ]
                    }
                ]
            },
            {
                'wk_range': 'Week 9-12',
                'days': [
                    {
                        'num': 1,
                        'focus': 'Arms',
                        'exs': [
                            {'name': 'Bicep Curls', 'type': 'Strength', 'sets': 5, 'reps': 10},
                            {'name': 'Tricep Extensions', 'type': 'Strength', 'sets': 5, 'reps': 10},
                            {'name': 'Hammer Curls', 'type': 'Strength', 'sets': 5, 'reps': 10}
             

In [116]:
plan = workout.WorkoutPlan(**response_json["WorkoutPlan"])

print(plan)

wks=[Wk(wk_range='Week 1-4', days=[Day(num=1, focus='Arms', exs=[Ex(name='Bicep Curls', type='Strength', sets='3', 
reps='10', dur=None), Ex(name='Tricep Extensions', type='Strength', sets='3', reps='10', dur=None), Ex(name='Hammer
Curls', type='Strength', sets='3', reps='10', dur=None)]), Day(num=2, focus='Chest and Triceps', 
exs=[Ex(name='Bench Press', type='Strength', sets='4', reps='8', dur=None), Ex(name='Tricep Dips', type='Strength',
sets='3', reps='10', dur=None), Ex(name='Push-Ups', type='Strength', sets='3', reps='10', dur=None)]), Day(num=3, 
focus='Back and Biceps', exs=[Ex(name='Deadlifts', type='Strength', sets='4', reps='8', dur=None), 
Ex(name='Pull-Ups', type='Strength', sets='3', reps='10', dur=None), Ex(name='Barbell Curls', type='Strength', 
sets='3', reps='10', dur=None)]), Day(num=4, focus='Legs and Shoulders', exs=[Ex(name='Squats', type='Strength', 
sets='4', reps='8', dur=None), Ex(name='Lunges', type='Strength', sets='3', reps='10', dur=None), Ex(name='Shoulder
Press', type='Strength', sets='3', reps='10', dur=None)])])] notes=[N(content='Increase the weight and intensity 
gradually over the 3-month duration.'), N(content='Incorporate kettlebell and resistance band exercises to add 
variety to the workout program.')]